In [5]:
from roboflow import Roboflow
rf = Roboflow(api_key="dTtRVrZhh5aMgZfDZdWb")
project = rf.workspace("hcmus-zhtrt").project("disease-plant-leaf")
version = project.version(1)
dataset = version.download("folder")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Disease-Plant-Leaf-1 in folder:: 100%|██████████| 10003/10003 [00:07<00:00, 1315.82it/s]


In [11]:
import os
import shutil
import random

# Original dataset with multiple classes
dataset_path = "Disease-Plant-Leaf-1/train"

# Output dataset (binary classes)
output_path = "PotatoDataset"
classes = ["Potato", "Not_Potato"]

# Train/Val/Test split ratios
train_ratio, val_ratio, test_ratio = 0.7, 0.2, 0.1

# Create output dirs
for split in ["train", "val", "test"]:
    for cls in classes:
        os.makedirs(os.path.join(output_path, split, cls), exist_ok=True)

# Function to check if class is potato
def is_potato(class_name):
    return class_name.lower() == "potato"

# Process each class in original dataset
for class_name in os.listdir(dataset_path):
    class_dir = os.path.join(dataset_path, class_name)
    if not os.path.isdir(class_dir):
        continue
    
    # Map to potato / not_potato
    target_class = "potato" if is_potato(class_name) else "not_potato"

    # Shuffle images
    images = os.listdir(class_dir)
    random.shuffle(images)

    n = len(images)
    train_end = int(train_ratio * n)
    val_end = train_end + int(val_ratio * n)

    for i, img in enumerate(images):
        src = os.path.join(class_dir, img)
        if i < train_end:
            dst = os.path.join(output_path, "train", target_class, img)
        elif i < val_end:
            dst = os.path.join(output_path, "val", target_class, img)
        else:
            dst = os.path.join(output_path, "test", target_class, img)
        shutil.copy(src, dst)

print("✅ Dataset preprocessing complete! Check 'PotatoDataset/'")


✅ Dataset preprocessing complete! Check 'PotatoDataset/'


In [12]:
import os
from PIL import Image

# Paths
root_dir = "Disease-Plant-Leaf-1/train"  # replace with your dataset path
img_size = (224, 224)  # YOLO cls default

def resize_images(folder):
    for subdir, _, files in os.walk(folder):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(subdir, file)
                try:
                    img = Image.open(img_path).convert("RGB")
                    img = img.resize(img_size)
                    img.save(img_path)  # overwrite
                except Exception as e:
                    print(f"Error resizing {img_path}: {e}")

# Apply to all sets
for split in ["train", "valid", "test"]:
    resize_images(os.path.join(root_dir, split))

print("✅ All images resized successfully!")


✅ All images resized successfully!


In [13]:
from ultralytics import YOLO

# load classification model (ResNet-like backbone)
model = YOLO("yolov8n-cls.pt")

# train
model.train(
    data="Disease-Plant-Leaf-1",
    epochs=25,
    imgsz=224,
    batch=16
)


Ultralytics 8.3.201  Python-3.11.4 torch-2.8.0+cpu CPU (AMD Ryzen 5 5600H with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Disease-Plant-Leaf-1, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=Tru

KeyboardInterrupt: 

In [7]:
# Validate on test set
metrics = model.val(data="Disease-Plant-Leaf-1/test")
print(metrics)


Ultralytics 8.3.201  Python-3.11.4 torch-2.8.0+cpu CPU (AMD Ryzen 5 5600H with Radeon Graphics)
YOLOv8n-cls summary (fused): 30 layers, 1,448,971 parameters, 0 gradients, 3.3 GFLOPs
WARNING Dataset 'split=train' not found at D:\USER\OneDrive\Desktop\Final_yr\notebook\Disease-Plant-Leaf-1\test\train
Found 1993 images in subdirectories. Attempting to split...
Splitting D:\USER\OneDrive\Desktop\Final_yr\notebook\Disease-Plant-Leaf-1\test (11 classes, 1993 images) into 80% train, 20% val...
Split complete in D:\USER\OneDrive\Desktop\Final_yr\notebook\Disease-Plant-Leaf-1\test_split 
train: D:\USER\OneDrive\Desktop\Final_yr\notebook\Disease-Plant-Leaf-1\test_split\train... found 1590 images in 11 classes  
val: D:\USER\OneDrive\Desktop\Final_yr\notebook\Disease-Plant-Leaf-1\test_split\val... found 403 images in 11 classes  
test: None...
val: Fast image access  (ping: 0.10.0 ms, read: 7.23.1 MB/s, size: 51.4 KB)
val: Scanning D:\USER\OneDrive\Desktop\Final_yr\notebook\Disease-Plant-Leaf-1\t

In [9]:
# Predict on a single image
results = model.predict("Disease-Plant-Leaf-1/test/Apple/00907d8b-6ae6-4306-bfd7-d54471981a86___RS_HL-5709_flipTB_JPG.rf.c3d515ab7c1d841877941b2e939de2b6.jpg")
print(results[0].probs)  # probabilities per class



image 1/1 d:\USER\OneDrive\Desktop\Final_yr\notebook\Disease-Plant-Leaf-1\test\Apple\00907d8b-6ae6-4306-bfd7-d54471981a86___RS_HL-5709_flipTB_JPG.rf.c3d515ab7c1d841877941b2e939de2b6.jpg: 224x224 Apple 1.00, Soybean 0.00, Cherry 0.00, Raspberry 0.00, Peach 0.00, 10.3ms
Speed: 3.8ms preprocess, 10.3ms inference, 0.4ms postprocess per image at shape (1, 3, 224, 224)
ultralytics.engine.results.Probs object with attributes:

data: tensor([1.0000e+00, 1.2026e-10, 1.2283e-09, 1.8441e-10, 2.9901e-10, 2.8658e-12, 1.3559e-10, 1.1366e-09, 6.5091e-09, 2.5249e-11, 1.6442e-11])
orig_shape: None
shape: torch.Size([11])
top1: 0
top1conf: tensor(1.)
top5: [0, 8, 2, 7, 4]
top5conf: tensor([1.0000e+00, 6.5091e-09, 1.2283e-09, 1.1366e-09, 2.9901e-10])
